<a href="https://colab.research.google.com/github/nicolinhainc/mypipeline/blob/master/CNNbestsofar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%cd '/content'

/content


In [0]:
!wget --no-check-certificate 'https://www.dropbox.com/s/nw79ua6y1nng9s4/png.zip?dl=1' -O ./png.zip


--2020-04-22 03:24:26--  https://www.dropbox.com/s/nw79ua6y1nng9s4/png.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/nw79ua6y1nng9s4/png.zip [following]
--2020-04-22 03:24:26--  https://www.dropbox.com/s/dl/nw79ua6y1nng9s4/png.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf49232f2e559a4c424988f3db3.dl.dropboxusercontent.com/cd/0/get/A2Qa4-mii0qKWP0CkuJAR3a1VnMu9fEMZPKShi7yeCcVA5hKyaje0a1rwA7WYyqmei6KUXmKbDUiGapvgD0W8Twzk-8QaatoyzT1JY4YarixwKgdMjE72PbvgVWz00gVliM/file?dl=1# [following]
--2020-04-22 03:24:27--  https://ucf49232f2e559a4c424988f3db3.dl.dropboxusercontent.com/cd/0/get/A2Qa4-mii0qKWP0CkuJAR3a1VnMu9fEMZPKShi7yeCcVA5hKyaje0a1rwA7WYyqmei6KUXmKbDUiGapvgD0W8Twzk-8QaatoyzT1JY4YarixwKgdMjE72PbvgVWz00g

In [2]:
!unzip -q ./adnitest101050prep.zip


replace __MACOSX/._adnitest101050prep? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [0]:
# Cell #7
# now we are ready to load data and train classifier
# while we did this in separate cells before, we will do it all in one
# this should look familiar.
import numpy as np
import tensorflow as tf
!pip install tensornets
import tensornets as nets
import tensorflow_hub as hub
import keras
import tensorflow.keras 
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Flatten,Dropout
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tensorflow.keras.applications.nasnet import NASNetLarge, NASNetMobile, preprocess_input
directory = "./content"

base_model=tensorflow.keras.applications.nasnet.NASNetLarge(input_shape=(331, 331, 3), include_top=False, weights='imagenet', input_tensor=None, pooling=None) #imports the nasnet large model and discards the last 1000 neuron layer.

# diesen Teil würde ich gleich hier machen, weil die Layer unten willst du ja auch jeden fall trainable haben.
# Es geht whrs darum, die Layer von deinem TransferLearning Model trainable zu machen oder?
# Dann einfach aus model --> base_model machen:

for i,layer in enumerate(base_model.layers):
  print(i,layer.name)

for layer in base_model.layers:
    layer.trainable=False
# or if we want to set the first 20 layers of the network to be non-trainable
#for layer in base_model.layers[:1039]:
#    layer.trainable=False
#for layer in base_model.layers[1039:]:
#    layer.trainable=True


x=base_model.output
x=GlobalAveragePooling2D()(x)
#x=Dropout(0.5)(x)
x=Dense(1000,activation='relu')(x) #dense layer 2
x=Dropout(0.5)(x)
x=Dense(1000,activation='relu')(x) #dense layer 3
x=Dropout(0.5)(x)
preds=Dense(2,activation='softmax')(x) #final layer with softmax activation, for 2 classes

# Also ich verwende als ClassificationLayer immer sigmoid bei binary Classification.
# Der Wert der dann raus kommt, ist einfach die Wahrscheinlichkeit: 
# Ergebnis 0.0 --> ganz sicher Klasse 0
# Ergebnis 1.0 --> ganz sicher Klasse 1
# Ergebnis 0.5 --> ganz unsicher welche Klasse.

#preds=Dense(1,activation='sigmoid')(x)



model=Model(inputs=base_model.input,outputs=preds) # to load saved model with weights use model= tensorflow.keras.models.load_model('myLGGtestmodel.h5')
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture



train_datagen=ImageDataGenerator(rotation_range=30, zoom_range=0.50, width_shift_range=0.4, height_shift_range=0.4, shear_range=0, fill_mode='nearest', horizontal_flip=False, data_format='channels_last', validation_split=0.0, dtype='float32') #included in our dependencies
# add the directory below very important ##########################

train=train_datagen.flow_from_directory('./adnitest101050prep/train',
                                                 target_size=(331,331),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

val_datagen=ImageDataGenerator()

val=val_datagen.flow_from_directory('./adnitest101050prep/validate',
                                                 target_size=(331,331),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=False)

# auch hier, wenn du nur 2 Klassen hast, dann nimmt man eig binary_crossentropy
#model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])


# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train.n//train.batch_size
step_size_val=val.n//val.batch_size
#model.fit_generator(generator=train_generator,
#                   steps_per_epoch=step_size_train,
#                   epochs=3)

history = model.fit(train, steps_per_epoch=step_size_train, epochs=5,
                              validation_data=val, validation_steps=step_size_val, 
                              verbose=1)       


0 input_6
1 stem_conv1
2 stem_bn1
3 activation_1300
4 reduction_conv_1_stem_1
5 reduction_bn_1_stem_1
6 activation_1301
7 activation_1303
8 separable_conv_1_pad_reduction_left1_stem_1
9 separable_conv_1_pad_reduction_right1_stem_1
10 separable_conv_1_reduction_left1_stem_1
11 separable_conv_1_reduction_right1_stem_1
12 separable_conv_1_bn_reduction_left1_stem_1
13 separable_conv_1_bn_reduction_right1_stem_1
14 activation_1302
15 activation_1304
16 separable_conv_2_reduction_left1_stem_1
17 separable_conv_2_reduction_right1_stem_1
18 activation_1305
19 separable_conv_2_bn_reduction_left1_stem_1
20 separable_conv_2_bn_reduction_right1_stem_1
21 separable_conv_1_pad_reduction_right2_stem_1
22 activation_1307
23 reduction_add_1_stem_1
24 separable_conv_1_reduction_right2_stem_1
25 separable_conv_1_pad_reduction_right3_stem_1
26 activation_1309
27 separable_conv_1_bn_reduction_right2_stem_1
28 separable_conv_1_reduction_right3_stem_1
29 separable_conv_1_reduction_left4_stem_1
30 activation_

In [9]:
# this cell is to predict image classes

predictions=model.predict(val, verbose=1, steps=step_size_val, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False)

 

UnidentifiedImageError: ignored

In [0]:
print([sum(x)/len(x) for x in zip(*predictions)])
#this gets you the averages for each position in the array!!!

In [0]:
np.mean(predictions)

0.5